In [ ]:
import os
import numpy as np
import pandas as pd
from glob import glob
import cv2
import torch
from torch.utils import data
from PIL import Image
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
txt_train = np.genfromtxt('train/gt_train.txt', delimiter =';', dtype = None, encoding = None)

#creating a dictionary with image names as key and annotations as value
dic_train ={}
for i in range (0, len(txt_train)):
    
    #image name is first element of annotation file
    img_name = txt_train[i][0]
    #4 coordinates
    target = [txt_train[i][1], txt_train[i][2], txt_train[i][3], txt_train[i][4], txt_train[i][5]]
    #last element is the class number
    cls = txt_train[i][-1]
    #if multiple objects, store coordinates and classes as list of lists
    if(img_name in dic_train):
        dic_train[img_name].append(target)
    else:
        dic_train[img_name] = [target]

print("Number of Images: " + str(len(dic_train)))

In [ ]:
txt_valid = np.genfromtxt('valid/gt_valid.txt', delimiter =';', dtype = None, encoding = None)

#creating a dictionary with image names as key and annotations as value
dic_valid ={}
for i in range (0, len(txt_valid)):

    #image name is first element of annotation file
    img_name = txt_valid[i][0]
    #4 coordinates
    target = [txt_valid[i][1], txt_valid[i][2], txt_valid[i][3], txt_valid[i][4], txt_valid[i][5]]
    #last element is the class number
    cls = txt_valid[i][-1]
    #if multiple objects, store coordinates and classes as list of lists
    if(img_name in dic_valid):
        dic_valid[img_name].append(target)
    else:
        dic_valid[img_name] = [target]

print("Number of Images: " + str(len(dic_valid)))

In [ ]:
class myDataset(torch.utils.data.Dataset):

    def __init__(self, root, _dic, transforms = None):

        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.dic = _dic
 
 
    def __getitem__(self, idx):

        img_path = os.path.join(self.root, "images", self.imgs[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #convert the numpy array to PIL Image
        img = Image.fromarray(img)
        #get objects in the image
        objects = self.dic[self.imgs[idx]]
        #get bounding box coordinates for each object in image
        boxes = []
        labels = []
        for obj in objects:
            name = obj[-1]
            labels.append(np.int(name))
            #get bounding box coordinates
            xmin = np.float(obj[0])
            ymin = np.float(obj[1])
            xmax = np.float(obj[2])
            ymax = np.float(obj[3])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype = torch.float32)
        labels = torch.as_tensor(labels, dtype = torch.int64)        
 
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        #suppose all instances are not crowd
        iscrowd = torch.zeros((len(objects),), dtype = torch.int64)
 
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
 
        if self.transforms is not None:
            img, target = self.transforms(img, target)
 
        return img, target


    def __len__(self):
        return len(self.imgs)

In [ ]:
import utilss
import transforms as T
from engine import train_one_epoch, evaluate
 
def get_transform(train):

    transforms = []
    #converts a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    
    return T.Compose(transforms)

In [ ]:
from engine import train_one_epoch, evaluate
import utilss
import torch.nn as nn
os.environ['TORCH_HOME'] = ''

root_train = r'train'
root_valid = r'valid'

#train on the GPU if available else CPU.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#44 classes = 43 + background
num_classes = 44
#send the data to the myDataset class (Apply transformations, Get bbox, labels, objects)
dataset_train = myDataset(root_train, _dic = dic_train, transforms = get_transform(train = True))
dataset_valid = myDataset(root_valid, _dic = dic_valid, transforms = get_transform(train = False))

#define training and validation data loaders
data_loader = torch.utils.data.DataLoader(dataset_train, batch_size = 2, shuffle = True, collate_fn = utilss.collate_fn)
data_loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size = 2, shuffle = False, collate_fn = utilss.collate_fn)

# Define model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = False, progress = True, num_classes = num_classes, pretrained_backbone = True)

#move the model to device
model.to(device)

print("Model loaded")

In [ ]:
import pickle

def save_all(epoch, losses, loss_box_reg, loss_rpn_box_reg, loss_classifier, loss_objectness, stat0, stat1, stat2, stat3,
        stat4, stat5, stat6, stat7, stat8, stat9, stat10, stat11, optimizer):

    with open(f'varsTest{epoch}.pickle', 'wb') as f:
        pickle.dump([losses, loss_box_reg, loss_rpn_box_reg, loss_classifier, loss_objectness, stat0, stat1, stat2, stat3,
        stat4, stat5, stat6, stat7, stat8, stat9, stat10, stat11], f)

    torch.save(model, f'trainTest{epoch}.pkl')

    torch.save(model.state_dict(), f'trainTest{epoch}.pth')
    torch.save({
        'epoch' : epoch,
        "model_state_dict" : model.state_dict(),
        'optimizer_state_dict' : optimizer.state_dict(),
    }, f'ckptTest{epoch}.pth') 

In [ ]:
from engine import train_one_epoch, evaluate
import utilss
from IPython.display import clear_output
import pickle

#constructing the optimizer
params = [p for p in model.parameters() if p.requires_grad]

# SGD
optimizer = torch.optim.SGD(params, lr = 0.0005, momentum = 0.9, weight_decay = 0.0005)

#learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 1, T_mult = 2)

num_epochs = 1000
losses = []
loss_box_reg = []
loss_rpn_box_reg = []
loss_classifier = []
loss_objectness = []

stat0 = []
stat1 = []
stat2 = []
stat3 = []
stat4 = []
stat5 = []
stat6 = []
stat7 = []
stat8 = []
stat9 = []
stat10 = []
stat11 = []

for epoch in range(num_epochs):

    metrics = train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq = 1000)
    losses.append(float(str(metrics.meters['loss']).split(" ")[0]))
    loss_box_reg.append(float(str(metrics.meters['loss_box_reg']).split(" ")[0]))
    loss_rpn_box_reg.append(float(str(metrics.meters['loss_rpn_box_reg']).split(" ")[0]))
    loss_classifier.append(float(str(metrics.meters['loss_classifier']).split(" ")[0]))
    loss_objectness.append(float(str(metrics.meters['loss_objectness']).split(" ")[0]))

    #update the learning rate
    lr_scheduler.step()

    #evaluate on the test dataset
    _, metric_logger = evaluate(model, data_loader_valid, device=device)

    stat = _.coco_eval['bbox'].stats
    
    #append all stats
    stat0.append(stat[0])
    stat1.append(stat[1])
    stat2.append(stat[2])
    stat3.append(stat[3])
    stat4.append(stat[4])
    stat5.append(stat[5])
    stat6.append(stat[6])
    stat7.append(stat[7])
    stat8.append(stat[8])
    stat9.append(stat[9])
    stat10.append(stat[10])
    stat11.append(stat[11])
    
    #save model after each 100 epochs
    if epoch % 100 == 0:
        save_all(epoch, losses, loss_box_reg, loss_rpn_box_reg, loss_classifier, loss_objectness, stat0, stat1, stat2, stat3,
        stat4, stat5, stat6, stat7, stat8, stat9, stat10, stat11, optimizer)
    
    print('')
    print('==================================================')
    print('')

print("Done!")

In [ ]:
r,c = 9,2
fig, ax = plt.subplots(nrows=r, ncols=c)
fig.set_figheight(40)
fig.set_figwidth(10)
fig.subplots_adjust(left=14,right=15, top=6, bottom=5, hspace=1, wspace=1)


ax1 = plt.subplot(r, c, 1)
ax1.set_title("Losses")
ax2 = plt.subplot(r, c, 2)
ax2.set_title("Loss Box Reg")
ax3 = plt.subplot(r, c, 3)
ax3.set_title("Loss RPN Box Reg")
ax4 = plt.subplot(r, c, 4)
ax4.set_title("Loss Classifier")
ax5 = plt.subplot(r, c, 5)
ax5.set_title("Loss Objectness")
ax6 = plt.subplot(r, c, 6)
ax6.set_title("(AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100")
ax7 = plt.subplot(r, c, 7)
ax7.set_title("(AP) @[ IoU=0.50      | area=   all | maxDets=100")
ax8 = plt.subplot(r, c, 8)
ax8.set_title("(AP) @[ IoU=0.75      | area=   all | maxDets=100")
ax9 = plt.subplot(r, c, 9)
ax9.set_title("(AP) @[ IoU=0.50:0.95 | area= small | maxDets=100")
ax10 = plt.subplot(r, c, 10)
ax10.set_title("(AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100")
ax11 = plt.subplot(r, c, 11)
ax11.set_title("(AP) @[ IoU=0.50:0.95 | area= large | maxDets=100")
ax12 = plt.subplot(r, c, 12)
ax12.set_title("(AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1")
ax13 = plt.subplot(r, c, 13)
ax13.set_title("(AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10")
ax14 = plt.subplot(r, c, 14)
ax14.set_title("(AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100")
ax15 = plt.subplot(r, c, 15)
ax15.set_title("(AR) @[ IoU=0.50:0.95 | area= small | maxDets=100")
ax16 = plt.subplot(r, c, 16)
ax16.set_title("(AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100")
ax17 = plt.subplot(r, c, 17)
ax17.set_title("(AR) @[ IoU=0.50:0.95 | area= large | maxDets=100")

ax1.plot(losses, 'b')
ax2.plot(loss_box_reg, 'b')
ax3.plot(loss_rpn_box_reg, 'b')
ax4.plot(loss_classifier, 'b')
ax5.plot(loss_objectness, 'b')
ax1.plot(losses, 'b')
ax2.plot(loss_box_reg, 'b')
ax3.plot(loss_rpn_box_reg, 'b')
ax4.plot(loss_classifier, 'b')
ax5.plot(loss_objectness, 'b')
ax6.plot(stat0, 'b')
ax7.plot(stat1, 'b')
ax8.plot(stat2, 'b')
ax9.plot(stat3, 'b')
ax10.plot(stat4, 'b')
ax11.plot(stat5, 'b')
ax12.plot(stat6, 'b')
ax13.plot(stat7, 'b')
ax14.plot(stat8, 'b')
ax15.plot(stat9, 'b')
ax16.plot(stat10, 'b')
ax17.plot(stat11, 'b')
plt.show()

In [ ]:
import pickle

with open('vars5.pickle', 'rb') as f:
    x = pickle.load(f)